In [1]:
import sys
import tensorflow as tf
sys.path.append('../../generative-adversarial-nets/')
from ganetwork import GAN, CGAN
from sklearn import datasets
from sklearn.preprocessing import minmax_scale, LabelBinarizer

In [2]:
ds = datasets.load_iris()
X, y = minmax_scale(ds.data), LabelBinarizer().fit_transform(ds.target)

In [3]:
discriminator_layers=[(4, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(6, None), (5, tf.nn.tanh), (4, None)]
gan = GAN(discriminator_layers, generator_layers)
gan.train(X, nb_epoch=50, batch_size=5)

Epoch: 0, discriminator loss: 1.6425416469573975, generator loss: 0.3051425814628601
Epoch: 1, discriminator loss: 1.5637255907058716, generator loss: 0.3560844957828522
Epoch: 2, discriminator loss: 1.4890048503875732, generator loss: 0.4240081310272217
Epoch: 3, discriminator loss: 1.4099189043045044, generator loss: 0.4973537027835846
Epoch: 4, discriminator loss: 1.415370225906372, generator loss: 0.550670862197876
Epoch: 5, discriminator loss: 1.4028018712997437, generator loss: 0.5988228917121887
Epoch: 6, discriminator loss: 1.401627779006958, generator loss: 0.6238316893577576
Epoch: 7, discriminator loss: 1.3775824308395386, generator loss: 0.5924467444419861
Epoch: 8, discriminator loss: 1.4147884845733643, generator loss: 0.6264958381652832
Epoch: 9, discriminator loss: 1.4046789407730103, generator loss: 0.6062267422676086
Epoch: 10, discriminator loss: 1.414916753768921, generator loss: 0.6359939575195312
Epoch: 11, discriminator loss: 1.3779041767120361, generator loss: 0

In [4]:
discriminator_layers=[(7, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(9, None), (5, tf.nn.tanh), (4, None)]
cgan = CGAN(discriminator_layers, generator_layers)
cgan.train(X, y, nb_epoch=50, batch_size=5)

Epoch: 0, discriminator loss: 1.585523247718811, generator loss: 0.3390282690525055
Epoch: 1, discriminator loss: 1.5180755853652954, generator loss: 0.3921923041343689
Epoch: 2, discriminator loss: 1.466070294380188, generator loss: 0.44621700048446655
Epoch: 3, discriminator loss: 1.4237420558929443, generator loss: 0.49713078141212463
Epoch: 4, discriminator loss: 1.3976385593414307, generator loss: 0.5310805439949036
Epoch: 5, discriminator loss: 1.3913769721984863, generator loss: 0.5751379132270813
Epoch: 6, discriminator loss: 1.3661918640136719, generator loss: 0.5927526354789734
Epoch: 7, discriminator loss: 1.3652268648147583, generator loss: 0.6385265588760376
Epoch: 8, discriminator loss: 1.3417384624481201, generator loss: 0.6637088656425476
Epoch: 9, discriminator loss: 1.3709561824798584, generator loss: 0.676974892616272
Epoch: 10, discriminator loss: 1.4059549570083618, generator loss: 0.6972790956497192
Epoch: 11, discriminator loss: 1.4187963008880615, generator loss